In [1]:
import numpy as np

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [3]:
572/4

143.0

In [6]:
576/2

288.0

In [18]:
def unet(input_img):
    # padding = 0 , stride = 1
    # input image = 576*576
    c1 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(input_img) #576*576
    c2 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(c1) # 576*576
    mp1 = MaxPool2D(pool_size=(2,2))(c2) # 288*288
    c3 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(mp1) # 288*288
    c4 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(c3) # 288*288
    mp2 = MaxPool2D(pool_size=(2,2))(c4) # 144*144
    c5 = Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')(mp2) # 144*144
    c6 = Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')(c5) # 144*144
    mp3 = MaxPool2D(pool_size=(2,2))(c6) # 72*72
    c7 = Conv2D(filters=512, kernel_size=(3,3), activation='relu', padding='same')(mp3) # 72*72
    c8 = Conv2D(filters=512, kernel_size=(3,3), activation='relu', padding='same')(c7) # 72*72
    mp4 = MaxPool2D(pool_size=(2,2))(c8) # 36*36
    dr1 = Dropout(rate=0.3)(mp4)
    c9 = Conv2D(filters=1024, kernel_size=(3,3), activation='relu', padding='same')(dr1) # 36*36
    c10 = Conv2D(filters=1024, kernel_size=(3,3), activation='relu', padding='same')(c9) # 36*36
    
    #upsampling followed by a 2x2 convolution
#     up1 = UpSampling2D(size=(2,2))(c10) # 56*56
    # up convolve
    up1 = Conv2DTranspose(512, (2,2), strides=(2,2), padding='valid')(c10)    # 72*72 concatenate c8
    merge1 = concatenate([c8, up1]) # 72*72
    c11 = Conv2D(filters=512, kernel_size=(3,3), activation='relu', padding='same')(merge1) # 72*72
    c12 = Conv2D(filters=512, kernel_size=(3,3), activation='relu', padding='same')(c11) # 72*72
#     up2 = UpSampling2D(size=(2,2))(c12) # 104*104
    up2 = Conv2DTranspose(256, (2,2), strides=(2,2), padding='valid')(c12) # 144*144

    # concatenate c6
    merge2 = concatenate([c6, up2]) # 144*144
    c13 = Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')(merge2) # 144*144
    c14 = Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')(c13) # 144*144
    up3 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='valid')(c14) # 288*288
#     # concatenate c4
    merge3 = concatenate([c4, up3])
    c15 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(merge3) # 288*288
    c16 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(c15) # 288*288
    up4 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='valid')(c16) # 576*576
#     # conctenate c2
    merge4 = concatenate([c2, up4]) # 576*576
    c17 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(merge4) # 576*576
    c18 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(c17) # 576*576
    # filters here need to be the nu,ber of classes
    c19 = Conv2D(filters=1, kernel_size=(1,1), padding='same', activation='sigmoid')(c18) # 576*576
    
    return c19

In [21]:
input_img = Input(shape = (864, 576, 3))
model = Model(input_img, unet(input_img))

In [22]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 864, 576, 3) 0                                            
__________________________________________________________________________________________________
conv2d_83 (Conv2D)              (None, 864, 576, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_84 (Conv2D)              (None, 864, 576, 64) 36928       conv2d_83[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_20 (MaxPooling2D) (None, 432, 288, 64) 0           conv2d_84[0][0]                  
____________________________________________________________________________________________

In [15]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

    return 1 - (numerator + 1) / (denominator + 1)